<a href="https://colab.research.google.com/github/JosselinPerret/DataDrivenControl-EI/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,TimeDistributed
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df_in = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/EI Data driven control/bdd_in_mat_05.csv')
df_out = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/EI Data driven control/bdd_out_mat_05.csv')

In [ ]:
print(f"Dimension of df_in: {df_in.shape}")
print(f"Dimension of df_out: {df_out.shape}")

on se place dans le cas où la voiture est immobile, donc il suffit de regarder une seule hélice.

In [ ]:
# Extract the first temporal series (first row) from df_in
first_temporal_series_in = df_in.iloc[0]

# Extract the corresponding first temporal series from df_out
first_temporal_series_out = df_out.iloc[0]

# Convert to numpy arrays for plotting if preferred, or keep as pandas Series
first_temporal_series_in_array = first_temporal_series_in.to_numpy()
first_temporal_series_out_array = first_temporal_series_out.to_numpy()

print("First temporal series from df_in (first 10 values):")
print(first_temporal_series_in_array[:10])
print(f"\nLength of the df_in series: {len(first_temporal_series_in_array)}")

print("\nFirst temporal series from df_out (first 10 values):")
print(first_temporal_series_out_array[:10])
print(f"\nLength of the df_out series: {len(first_temporal_series_out_array)}")

# Plot both temporal series on the same graph
plt.figure(figsize=(15, 5))
plt.plot(first_temporal_series_in_array, label='df_in - Input Series')
plt.plot(first_temporal_series_out_array, label='df_out - Output Series')
plt.title('First Temporal Series (Input and Output)')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
for i in range(df_out.shape[0]):
    plt.plot(df_out.iloc[i], alpha=0.5, label=f'Row {i}')

plt.title('All Rows of df_out')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(first_temporal_series_in_array, first_temporal_series_out_array, alpha=0.5)
plt.title('Scatter Plot of First Temporal Series (df_in vs df_out)')
plt.xlabel('df_in Value')
plt.ylabel('df_out Value')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(first_temporal_series_in_array, bins=50, edgecolor='black')
plt.title('Distribution of Values in first_temporal_series_in_array')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

la sortie est l'accéleration verticale d'ou le fait que ce soit -g lorsque la commande est 0

l'entrée représente une commande normalisée surement le voltage

il faudra integrer 2x

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(first_temporal_series_out_array[:100])
plt.title('First 100 Values of first_temporal_series_out_array')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.grid(True)
plt.show()

il faut environ 10 points entre 40 et 50 (T_s = 0.05s) soit entre 2s et 2.5s --> soit 0.5/10 = T_s ça justifie le temps d'échantillonnage --> il faut vérifier sur toutes les donénes

on va faire un réseau de neurone pour prédire l'accéleration puis on intégrera 2 fois pour obtenir la position car notre capteur est un accelerometre

## Data Preparation

In [ ]:
import numpy as np
from scipy.stats import zscore

df_in_cleaned = df_in.copy()
df_out_cleaned = df_out.copy()

# Set the z_score_threshold
z_score_threshold = 3

z_scores_in = zscore(df_in_cleaned, axis=1, nan_policy='omit') # Calculate Z-scores for each row (time series)
abs_z_scores_in = np.abs(z_scores_in)

z_scores_out = zscore(df_out_cleaned, axis=1, nan_policy='omit')
abs_z_scores_out = np.abs(z_scores_out)

outlier_rows_in = (abs_z_scores_in > z_score_threshold).any(axis=1)
outlier_rows_out = (abs_z_scores_out > z_score_threshold).any(axis=1)

# Combine the identified outlier rows from both DataFrames
total_outlier_rows = outlier_rows_in | outlier_rows_out
# Create a boolean mask rows_to_keep by negating the total_outlier_rows
rows_to_keep = ~total_outlier_rows

# Filter both df_in_cleaned and df_out_cleaned using rows_to_keep
df_in_cleaned = df_in_cleaned[rows_to_keep]
df_out_cleaned = df_out_cleaned[rows_to_keep]

print(f"Original shape of df_in: {df_in.shape}")
print(f"Original shape of df_out: {df_out.shape}")
print(f"Number of rows identified as outliers: {np.sum(total_outlier_rows)}")
print(f"Shape of cleaned df_in_cleaned: {df_in_cleaned.shape}")
print(f"Shape of cleaned df_out_cleaned: {df_out_cleaned.shape}")

In [ ]:
plt.figure(figsize=(15, 7))
for i in range(df_out_cleaned.shape[0]):
    plt.plot(df_out_cleaned.iloc[i], alpha=0.5, label=f'Row {i}')

plt.title('All Rows of df_out_cleaned')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.grid(True)
plt.show()

In [ ]:
# Re-scale Input Data using the cleaned df_in_cleaned
X = df_in_cleaned.to_numpy()
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

print(f"\nOriginal shape of X after outlier removal: {X.shape}")
print(f"Shape of X_scaled after re-scaling: {X_scaled.shape}")

# Re-normalize Output Data using the cleaned df_out_cleaned
y = df_out_cleaned.to_numpy()
df_out_shifted = y - y[:, 0, np.newaxis]
global_max_abs_y = np.max(np.abs(df_out_shifted))
y_normalized = df_out_shifted / global_max_abs_y

print(f"\nOriginal shape of y after outlier removal: {y.shape}")
print(f"Shape of y_normalized after re-normalization: {y_normalized.shape}")
print(f"New global maximum absolute value for y normalization: {global_max_abs_y}")

In [ ]:
# Reshape both the normalized input (X) and normalized output (y) arrays into a 3D format (samples, timesteps, features)
# For this dataset, samples will be the number of rows, timesteps will be the number of columns, and features will be 1.

n_samples_cleaned, n_timesteps = X_scaled.shape
X_reshaped = X_scaled.reshape(n_samples_cleaned, n_timesteps, 1)

y_reshaped = y_normalized.reshape(n_samples_cleaned, n_timesteps, 1)

print(f"Original shape of X_scaled: {X_scaled.shape}")
print(f"Reshaped shape of X_reshaped: {X_reshaped.shape}")
print(f"Original shape of y_normalized: {y_normalized.shape}")
print(f"Reshaped shape of y_reshaped: {y_reshaped.shape}")

# Allocate 80% of the data for training and 20% for testing. Use train_test_split from sklearn.model_selection for this purpose.
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

### LSTM

In [ ]:
# --- Construction du modèle LSTM ---
model = Sequential()
model.add(LSTM(32, input_shape=(n_timesteps, 1), return_sequences=True))
model.add(Dense(1))  # prédiction par timestep

model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# --- Entraînement ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32
)

# --- Évaluation ---
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

In [ ]:
y_pred = model.predict(X_test)

g = 9.81

# Remettre à l’échelle d’origine
y_pred_real = y_pred * global_max_abs_y - g
y_test_real = y_test * global_max_abs_y - g

rmse = np.sqrt(mean_squared_error(y_test_real.flatten(), y_pred_real.flatten()))
print(f"RMSE: {rmse:.4f}")

In [ ]:
i = 1  # indice du sample à visualiser

fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot real and predicted output on the primary y-axis
ax1.plot(y_test_real[i].squeeze()[:200], label="Réel (Output)", color='blue')
ax1.plot(y_pred_real[i].squeeze()[:200], label="Prédit (Output)", color='red', linestyle='--')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Output Value (Acceleration)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a secondary y-axis for the input signal
ax2 = ax1.twinx()
ax2.plot(X_test[i].squeeze()[:200], label="Input Signal", color='green', alpha=0.7)
ax2.set_ylabel('Input Value (Normalized Command)', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Add title and legend
plt.title("Comparaison Input, sortie réelle vs prédite (Premiers 200 points)")
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
plt.grid(True)
plt.show()

In [ ]:
i = 1  # vous pouvez changer cet indice

# Données réelles
real_acceleration = y_test_real[i].squeeze()
# Données prédites
predicted_acceleration = y_pred_real[i].squeeze()

# Calcul de l'erreur d'accélération
error_acceleration = np.abs(real_acceleration - predicted_acceleration)

# Fonction pour calculer la position à partir de l'accélération
def calculate_position(acceleration, dt=0.05):
    velocity = np.cumsum(acceleration) * dt
    position = np.cumsum(velocity) * dt
    return position, velocity

# Calculer les positions et vitesses
real_position, real_velocity = calculate_position(real_acceleration)
predicted_position, predicted_velocity = calculate_position(predicted_acceleration)

# Affichage des résultats
plt.figure(figsize=(15, 12))

# Accélération
plt.subplot(4, 1, 1)
plt.plot(real_acceleration, label='Accélération réelle', color='blue')
plt.plot(predicted_acceleration, label='Accélération prédite', linestyle='--', color='red')
plt.legend()
plt.title('Comparaison des accélérations')
plt.grid(True)

# Erreur d'Accélération
plt.subplot(4, 1, 2)
plt.plot(error_acceleration, label='Erreur Absolue Accélération', linestyle=':', color='green')
plt.legend()
plt.title('Erreur Absolue d\'Accélération')
plt.grid(True)

# Vitesse
plt.subplot(4, 1, 3)
plt.plot(real_velocity, label='Vitesse réelle')
plt.plot(predicted_velocity, label='Vitesse prédite', linestyle='--')
plt.legend()
plt.title('Comparaison des vitesses')
plt.grid(True)

# Position
plt.subplot(4, 1, 4)
plt.plot(real_position, label='Position réelle')
plt.plot(predicted_position, label='Position prédite', linestyle='--')
plt.legend()
plt.title('Comparaison des positions')
plt.grid(True)

plt.tight_layout()
plt.show()

# Calculer l'erreur RMS pour chaque variable
def calculate_rmse(real, pred):
    return np.sqrt(np.mean((real - pred) ** 2))

print(f"RMSE Accélération: {calculate_rmse(real_acceleration, predicted_acceleration):.4f}")
print(f"RMSE Vitesse: {calculate_rmse(real_velocity, predicted_velocity):.4f}")
print(f"RMSE Position: {calculate_rmse(real_position, predicted_position):.4f}")

on a le modèle, on va comparer avec d'autres modèles

### SSM

In [ ]:
# find A B C

In [ ]:
import scipy.linalg

# 2. Define the discrete-time state-space matrices A and B
# State vector: [height, velocity]
# Equations:
# height(t+dt) = height(t) + velocity(t) * dt + 0.5 * acceleration(t) * dt^2
# velocity(t+dt) = velocity(t) + acceleration(t) * dt

dt = 0.05  # sampling time

A = np.array([[1, dt],
              [0, 1]])

B = np.array([[0.5 * dt**2],
              [dt]])

# 3. Define the Q and R matrices
# Q penalizes state deviations (height, velocity)
# R penalizes control input (acceleration)
Q = np.array([[10, 0],
              [0, 1]]) # Penalize height more than velocity
R = np.array([[0.1]]) # Penalize control input (acceleration) less

# 4. Calculate the optimal feedback gain matrix K
# Solve the Discrete Algebraic Riccati Equation (DARE) to find P
P = scipy.linalg.solve_discrete_are(A, B, Q, R)

# Calculate K using P
K = np.linalg.inv(R + B.T @ P @ B) @ (B.T @ P @ A)

print("State-space matrix A:\n", A)
print("\nState-space matrix B:\n", B)
print("\nCost matrix Q:\n", Q)
print("\nCost matrix R:\n", R)
print("\nOptimal feedback gain K:\n", K)

In [ ]:
import matplotlib.pyplot as plt

# 5. Set simulation parameters
target_height = 10.0  # meters
initial_height = 0.0  # meters
initial_velocity = 0.0  # m/s
simulation_duration = 20  # seconds

num_steps = int(simulation_duration / dt)

# 6. Initialize arrays to store results
height_history = np.zeros(num_steps)
velocity_history = np.zeros(num_steps)
acceleration_history = np.zeros(num_steps)

# Initial state
x_current = np.array([[initial_height], [initial_velocity]])

# 7. Implement simulation loop
for i in range(num_steps):
    # a. Calculate current state vector (already x_current)

    # b. Determine the error state (current state minus target state)
    # Target state: [target_height, 0]
    error_state = x_current - np.array([[target_height], [0.0]])

    # c. Compute the control input u (acceleration)
    u = -K @ error_state

    # d. Update the system's state using the discrete-time state-space equations
    x_current = A @ x_current + B @ u

    # e. Store the current height, velocity, and control input
    height_history[i] = x_current[0, 0]
    velocity_history[i] = x_current[1, 0]
    acceleration_history[i] = u[0, 0]

# Plotting the results
time = np.linspace(0, simulation_duration, num_steps)

plt.figure(figsize=(15, 10))

plt.subplot(3, 1, 1)
plt.plot(time, height_history)
plt.axhline(y=target_height, color='r', linestyle='--', label='Target Height')
plt.title('Height over Time')
plt.xlabel('Time (s)')
plt.ylabel('Height (m)')
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(time, velocity_history)
plt.title('Velocity over Time')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(time, acceleration_history)
plt.title('Controller Output (Acceleration) over Time')
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (m/s^2)')
plt.grid(True)

plt.tight_layout()
plt.show()

trop dur de trouver le modèle inverse donc on fait optimization based LSTM contrller